## Chinese-Alpaca-Plus-7B在C-Eval数据集上的解码示例

教程分为三部分：
1. 合并模型
2. 利用合并好的模型在C-Eval valid set上进行解码
3. 利用官方脚本测试效果

机器选择：此处使用A100进行预测（实际7B模型用V100应该也是可以的）。

In [ ]:
!nvidia-smi

Fri Jun 16 00:26:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 第一步：合并模型

这里不再一一介绍流程，感兴趣的可以去看另一个Colab Notebook：https://colab.research.google.com/drive/1FnFkyKhrnS7s-2lDDeous-AutdI_SkAd?usp=sharing

In [ ]:
!pip install torch==1.13.1
!pip install transformers==4.30.2
!pip install peft==0.3.0
!pip install sentencepiece
!git clone https://github.com/ymcui/Chinese-LLaMA-Alpaca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.

In [ ]:
!python ./Chinese-LLaMA-Alpaca/scripts/merge_llama_with_chinese_lora_low_mem.py \
    --base_model elinas/llama-7b-hf-transformers-4.29 \
    --lora_model ziqingyang/chinese-llama-plus-lora-7b,ziqingyang/chinese-alpaca-plus-lora-7b \
    --output_type huggingface \
    --output_dir alpaca-combined-hf

Base model: elinas/llama-7b-hf-transformers-4.29
LoRA model(s) ['ziqingyang/chinese-llama-plus-lora-7b', 'ziqingyang/chinese-alpaca-plus-lora-7b']:
Loading ziqingyang/chinese-llama-plus-lora-7b
Cannot find lora model on the disk. Downloading lora model from hub...
Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]




Fetching 7 files:  14% 1/7 [00:00<00:02,  2.36it/s]




















































































Fetching 7 files: 100% 7/7 [00:21<00:00,  3.07s/it]
Loading ziqingyang/chinese-alpaca-plus-lora-7b
Cannot find lora model on the disk. Downloading lora model from hub...
Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]
Fetching 7 files:  14% 1/7 [00:00<00:01,  5.46it/s]



Fetching 7 files:  43% 3/7 [00:00<00:00,  9.03it/s]















































































































Fetching 7 files: 100% 7/7 [00:28<00:00,  4.02s/it]
Cannot find lora model on the disk. Downloading lora model from hub

### 第二步：运行模型预测脚本，输出C-Eval valid set预测标签

相关教程：

In [ ]:
!cd ./Chinese-LLaMA-Alpaca/scripts/ceval && wget https://huggingface.co/datasets/ceval/ceval-exam/resolve/main/ceval-exam.zip && mkdir data && unzip ceval-exam.zip -d data

--2023-06-16 00:46:12--  https://huggingface.co/datasets/ceval/ceval-exam/resolve/main/ceval-exam.zip
Resolving huggingface.co (huggingface.co)... 13.224.81.102, 13.224.81.4, 13.224.81.43, ...
Connecting to huggingface.co (huggingface.co)|13.224.81.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/27/88/2788376e479829369a8f5cc54695d446426e8f192bcb541e2e898b8643652bf0/0e2a016ad940d1c8b3965767ac7bb33ea140f8ae6ca06d0fe38b0c8a5466ca7c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ceval-exam.zip%3B+filename%3D%22ceval-exam.zip%22%3B&response-content-type=application%2Fzip&Expires=1687135572&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzI3Lzg4LzI3ODgzNzZlNDc5ODI5MzY5YThmNWNjNTQ2OTVkNDQ2NDI2ZThmMTkyYmNiNTQxZTJlODk4Yjg2NDM2NTJiZjAvMGUyYTAxNmFkOTQwZDFjOGIzOTY1NzY3YWM3YmIzM2VhMTQwZjhhZTZjYTA2ZDBmZTM4YjBjOGE1NDY2Y2E3Yz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcm

In [ ]:
!ls ./Chinese-LLaMA-Alpaca/scripts/ceval/data

dev  test  val


In [ ]:
!cd ./Chinese-LLaMA-Alpaca/scripts/ceval && CUDA_VISIBLE_DEVICES=0 python eval.py \
    --model_path /content/alpaca-combined-hf \
    --cot False \
    --few_shot False \
    --with_prompt True \
    --constrained_decoding True \
    --temperature 0.2 \
    --n_times 1 \
    --ntrain 5 \
    --do_save_csv False \
    --do_test False \
    --output_dir /content/ceval-output

流式输出内容被截断，只能显示最后 5000 行内容。
 27% 6/22 [00:00<00:00, 16.53it/s]
=======begin 6=======
question:  下列四组矿质元素，哪组元素的缺素症表现为叶片缺绿?____
A. Fe，Cl，N，Ca
B. Mg，Fe，N，S
C. P，N，Mg，S
D. P，Mo，Mg，S
答案是什么？ 
response:  B
ans:  B
ground truth:  B 

=======end 6=======

=======begin 7=======
question:  在衰老的植物组织或器官中，蛋白质含量显著下降，其主要原因是____。
A. 蛋白质周转速率降低
B. 氨基酸的生物合成受阻
C. 蛋白质水解酶活性增加
D. 土壤中氮素含量下降
答案是什么？ 
response:  B
ans:  B
ground truth:  C 

=======end 7=======
 36% 8/22 [00:00<00:00, 16.48it/s]
=======begin 8=======
question:  G蛋白是由α、β、γ3种亚基构成的异三聚体，与GTP结合的活性位点在____上。
A. α亚基
B. β亚基
C. γ亚基
D. βγ亚基复合体
答案是什么？ 
response:  A
ans:  A
ground truth:  A 

=======end 8=======

=======begin 9=======
question:  已分化的细胞在特定情况下恢复分裂能力，重新进行细胞分裂的过程为____。
A. 细胞分化
B. 细胞分裂
C. 再分化
D. 脱分化
答案是什么？ 
response:  B
ans:  B
ground truth:  D 

=======end 9=======
 45% 10/22 [00:00<00:00, 16.45it/s]
=======begin 10=======
question:  植物器官、组织或细胞在形态结构、生化组成以及生理功能上的不对称性是____。
A. 细胞生长
B. 细胞分化
C. 极性
D. 细胞全能性
答案是什么？ 
response:  A
ans:  A
ground truth:  C 

### 第三步：查看预测结果

运行以下命令查看最终结果，json最后的ALL里会显示，这一次运行结果是：
```
  "All": {
    "score": 0.36701337295690933,
    "num": 1346,
    "correct": 494.0
  }
```

上述结果与我们论文中汇报的zero-shot 36.7（%）一致。需要注意的是解码存在随机性，如果希望多次运行可将`n_times`改为需要运行的次数。


In [ ]:
!cat ./ceval-output/take0/summary.json

{
  "law": {
    "score": 25.0,
    "num": 24,
    "correct": 6.0
  },
  "environmental_impact_assessment_engineer": {
    "score": 45.16129032258065,
    "num": 31,
    "correct": 14.0
  },
  "middle_school_biology": {
    "score": 47.61904761904762,
    "num": 21,
    "correct": 10.0
  },
  "college_chemistry": {
    "score": 29.166666666666668,
    "num": 24,
    "correct": 7.0
  },
  "college_economics": {
    "score": 34.54545454545455,
    "num": 55,
    "correct": 19.0
  },
  "middle_school_mathematics": {
    "score": 26.31578947368421,
    "num": 19,
    "correct": 4.999999999999999
  },
  "computer_architecture": {
    "score": 28.571428571428573,
    "num": 21,
    "correct": 6.0
  },
  "high_school_mathematics": {
    "score": 16.666666666666668,
    "num": 18,
    "correct": 3.0
  },
  "college_programming": {
    "score": 37.83783783783784,
    "num": 37,
    "correct": 14.0
  },
  "computer_network": {
    "score": 31.57894736842105,
    "num": 19,
    "correct": 6.0
  }